In [294]:
#data prep
#add rand into is_bad_word and word lengthto make better presentation
#take "on" and "hard" out of the profanity list
import pandas as pd
import random as rd
import math
df = pd.read_csv("visual.csv")
df.loc[df.word_corrected == 'on', 'is_bad_word'] = 0
df.loc[df.word_corrected == 'hard', 'is_bad_word'] = 0
df['profanity'] = df['is_bad_word'].map(lambda x: x + rd.random())
df['word length'] = df['word_length'].map(lambda x: x + 0.8*rd.random())
df['log frequency'] = df['frequency'].map(lambda x: math.log(x))
mini = abs(min(df['log frequency']))
df['log frequency'] = df['log frequency'] + mini
df = df[['word_corrected', 'year', 'log frequency','profanity','genre','word length']]

In [295]:
import plotly
plotly.tools.set_credentials_file(username='IreneZ', api_key='o3gHxCwNoSqvTcHQkKZc')

In [296]:
list(df.columns)

['word_corrected',
 'year',
 'log frequency',
 'profanity',
 'genre',
 'word length']

In [297]:
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF 

import pandas as pd
import time

In [298]:
dataset = df

#make grid
years_from_col = set(dataset['year'])
years_ints = sorted(list(years_from_col))
years = [str(year) for year in years_ints]

# make list of genres

genres = ['metal','rap','rock','country','blues']

columns = []
# make grid
for year in years:
    for genre in genres:
        dataset_by_year = dataset[dataset['year'] == int(year)]
        dataset_by_year_and_genre = dataset_by_year[dataset_by_year['genre'] == genre]
        for col_name in dataset_by_year_and_genre:
            # each column name is unique
            column_name = '{year}_{genre}_{header}_gapminder_grid'.format(
                year=year, genre=genre, header=col_name
            )
            a_column = Column(list(dataset_by_year_and_genre[col_name]), column_name)
            columns.append(a_column)

# upload grid
grid = Grid(columns)
url = py.grid_ops.upload(grid, 'gapminder_grid'+str(time.time()), auto_open=False)
url

'https://plot.ly/~IreneZ/115/'

In [299]:
#make figure

figure = {
    'data': [],
    'layout': {},
    'frames': [],
    'config': {'scrollzoom': True},
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [0, 12], 'title': 'Word Length', 'gridcolor': '#FFFFFF'}
figure['layout']['yaxis'] = {'range': [0, 2],'title': 'Profanity or Not', 'gridcolor': '#FFFFFF'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'

In [300]:
#add slider

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [301]:
#add play and pause
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

custom_colors = {
    'rap': 'rgb(255, 102, 102)',
    'rock': 'rgb(31, 122, 31)',
    'country': 'rgb(0, 128, 255)',
    'blues': 'rgb(255, 204, 0)',
    'metal': 'rgb(191, 0, 255)'
}

In [302]:
#fill in data and frames

col_name_template = '{year}_{genre}_{header}_gapminder_grid'
year = 1988
for genre in genres:
    data_dict = {
        'xsrc': grid.get_column_reference(col_name_template.format(
            year=year, genre=genre, header='word length'
        )),
        'ysrc': grid.get_column_reference(col_name_template.format(
            year=year, genre=genre, header='profanity'
        )),
        'mode': 'markers',
        'textsrc': grid.get_column_reference(col_name_template.format(
            year=year, genre=genre, header='word_corrected'
        )),
        'marker': {
            'sizemode': 'diameter',
            'sizeref': 0.02,
            'sizesrc': grid.get_column_reference(col_name_template.format(
                 year=year, genre=genre, header='log frequency'
            )),
            'color': custom_colors[genre]
        },
        'name': genre
    }
    figure['data'].append(data_dict)

In [303]:
#create frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for genre in genres:
        data_dict = {
        'xsrc': grid.get_column_reference(col_name_template.format(
            year=year, genre=genre, header='word length'
        )),
        'ysrc': grid.get_column_reference(col_name_template.format(
            year=year, genre=genre, header='profanity'
        )),
        'mode': 'markers',
        'textsrc': grid.get_column_reference(col_name_template.format(
            year=year, genre=genre, header='word_corrected'
        )),
        'marker': {
            'sizemode': 'diameter',
            'sizeref': 0.02,
            'sizesrc': grid.get_column_reference(col_name_template.format(
                 year=year, genre=genre, header='log frequency'
            )),
            'color': custom_colors[genre]
        },
        'name': genre
    }
        
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

In [304]:
py.icreate_animations(figure, 'gapminder_example'+str(time.time()))